# Data utilizada

Los datos utilizados hasta el momento corresponden al catalogo de estrellas variables de ASAS (ACVS), obtenidos de http://www.astrouw.edu.pl/asas/?page=download (Faltan los del catalogo general).

En la estructura de cada archivo se tienen varios sub-dataset con diferentes mediciones.
Dentro de los elementos importantes en la estructura de cada sub-dataset tenemos:
* ndata : Contiene el número de mediciones de cada sub dataset
* dataset: Identificador del dataset
* Ra y Dec: Son las coordenadas de cada dataset ( Ascensión recta y declinación respectivamente) 
* MAG: Es la magnitud y se posee una por cada apertura. Se tiene 5 magnitudes en total.
* MER: Es el frame error que describen la calidad fotométrica promedio de cada Frame. Uno por cada apertura.
* Frame: Número de frame.
* Grade: Calidad de la medición:
    * A - best data, no 29.999 (not measured) indication
    * B - mean data, no 29.999 (not measured) indication
    * C - A and B with 29.999 (not measured) indication
    * D - worst data, probably useless
 
Tambien en un futuro se pueden considerar datos de otros surveys como:
* Catalina Sky Survey-> http://nesssi.cacr.caltech.edu/DataRelease/


# Parser

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from scipy.optimize import curve_fit

In [2]:
listado = open("ejemplos/lista.txt","r")
files = listado.readlines()
listado.close()

In [9]:
#archivo = open("ejemplos/"+files[0].strip("\n"),"r")
archivo= open("000000-4411.0.txt","r")

In [10]:
text = archivo.readlines()
archivo.close()

In [11]:
text

['# The All Sky Automated Survey Data\n',
 '# gp@astrouw.edu.pl\n',
 '#\n',
 '# ################   DESCRIPTION   ###############\n',
 '# The ASAS Photometric Catalog is maintained separately for each\n',
 "# observed field, so for some stars independent 'datasets' of\n",
 '# measurements are available. Their mean magnitudes may slightly differ.\n',
 '#\n',
 "# In each 'dataset' (starting with #dataset=0,1,2,...):\n",
 "# 'desig' is ASAS designation (they may differ (by 1) at the last\n",
 '# digit of the RA & DEC fields\n',
 "# 'cra', 'cdec' are initial Catalog coordinates\n",
 "# 'ndata' is number of points in each dataset\n",
 "# 'cmag_*',  'cmer_*' are reference magnitude & dispersion for each aperture\n",
 "# 'nskip_*' is number of data points skipped when calculating 'cmag' & 'cmer'\n",
 "# 'ra', 'dec', 'mag', 'mer' are coordinates, magnitude and dispersion\n",
 '# calculated directly from the data\n',
 '#\n',
 '# Each data row consists of the following fields:\n',
 '# -  HJD-2450

## Funciones necesarias hacer el parseo de la data

In [12]:
def cut(c):
    flag = True
    data = []
    head = []
    patron = re.compile("\s+")
    for i in c:
        if flag == True:
            head.append(patron.split(i))
            flag = False
        else:
            data.append(patron.split(i))
    return np.array(data),head

In [16]:
def parser(data):
    c = []
    alpha = []
    gamma = []
    flag = 0
    for i in range(len(data)):
        flag1= re.findall("\# ######### LIGHT ",data[i])
        if flag1:
            data= data[i+1:]
            break
    
    for i in range(len(data)):
        head = re.findall("\A#\s+(.+)",data[i])
        new_line = re.findall("\A   (.+)",data[i]) 
        ra = re.findall("\#ra=\s+(.+)",data[i])
        dec = re.findall("\#dec=\s+(.+)",data[i])
        if ra:
            ra_aux = ra
        if dec:
            dec_aux = dec
        if head:
            [c.append(j) for j in head]
        if new_line:
            [c.append(k) for k in new_line]
            alpha.append(ra_aux)
            gamma.append(dec_aux)
            try:
                next_line = re.findall("\A   (.+)",data[i+1])
                if not next_line:
                    reg, col= cut(c)
                    if flag == 0:
                        df = pd.DataFrame(reg,columns=col)
                        df["RA"] = np.array(alpha)
                        df["DEC"] = np.array(gamma)
                        c = []
                        alpha = []
                        gamma = []
                        flag += 1                    
                    else:
                        df_aux = pd.DataFrame(reg,columns=col)
                        df_aux["RA"] = np.array(alpha)
                        df_aux["DEC"] = np.array(gamma)
                        df = pd.concat([df,df_aux],ignore_index=True)
                        c = []
                        alpha = []
                        gamma = []

            except:
                reg, col = cut(c)
                if flag == 0:
                    df = pd.DataFrame(reg,columns=col)
                    df["RA"] = np.array(alpha)
                    df["DEC"] = np.array(gamma)
                    c = []
                    alpha = []
                    gamma = []
                    flag += 1   
                else:    
                    df_aux = pd.DataFrame(reg,columns=col)
                    df_aux["RA"] = np.array(alpha)
                    df_aux["DEC"] = np.array(gamma)
                    df = pd.concat([df,df_aux],ignore_index=True)
    
    col[0].remove('GRADE')
    df[col[0]] = df[col[0]].astype(float)
    
    return df

dataset = parser(text)

#Problemas con el archivo 433 -> revisar

In [14]:
parser(text)

,HJD,MAG_0,MAG_1,MAG_2,MAG_3,MAG_4,MER_0,MER_1,MER_2,MER_3,MER_4,GRADE,FRAME,RA,DEC
0,1869.51295,13.276,13.304,13.288,13.260,13.286,0.046,0.031,0.021,0.022,0.023,A,189.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
1,1871.60896,13.305,13.287,13.302,13.310,13.301,0.062,0.034,0.024,0.023,0.024,A,508.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
2,1873.58380,13.264,13.184,13.177,13.164,13.160,0.075,0.038,0.027,0.028,0.029,A,820.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
3,1875.57591,13.215,13.197,13.248,13.248,13.203,0.068,0.036,0.025,0.025,0.026,A,1139.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
4,1879.56431,13.254,13.274,13.246,13.274,13.394,0.056,0.036,0.025,0.025,0.026,A,1769.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
5,1881.55358,13.285,13.283,13.335,13.488,13.481,0.054,0.034,0.024,0.023,0.026,A,2077.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
6,1885.54382,13.301,13.338,13.332,13.315,13.319,0.052,0.036,0.024,0.024,0.027,A,2384.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
7,1887.57123,13.027,13.069,13.014,13.010,13.020,0.053,0.032,0.023,0.023,0.025,A,2715.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
8,1904.53504,13.261,13.212,13.186,13.158,13.192,0.052,0.033,0.024,0.024,0.026,A,3785.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
9,1907.54992,13.239,13.247,13.322,13.344,13.276,0.038,0.032,0.021,0.020,0.022,A,4275.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4


In [17]:
dataset.head()

,HJD,MAG_0,MAG_1,MAG_2,MAG_3,MAG_4,MER_0,MER_1,MER_2,MER_3,MER_4,GRADE,FRAME,RA,DEC
0,1869.51295,13.276,13.304,13.288,13.260,13.286,0.046,0.031,0.021,0.022,0.023,A,189.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
1,1871.60896,13.305,13.287,13.302,13.310,13.301,0.062,0.034,0.024,0.023,0.024,A,508.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
2,1873.58380,13.264,13.184,13.177,13.164,13.160,0.075,0.038,0.027,0.028,0.029,A,820.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
3,1875.57591,13.215,13.197,13.248,13.248,13.203,0.068,0.036,0.025,0.025,0.026,A,1139.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4
4,1879.56431,13.254,13.274,13.246,13.274,13.394,0.056,0.036,0.025,0.025,0.026,A,1769.0,23.999703 23:59:58.9,-44.185656 -44:11:08.4


In [9]:
dataset.describe()

,HJD,MAG_0,MAG_1,MAG_2,MAG_3,MAG_4,MER_0,MER_1,MER_2,MER_3,MER_4,FRAME
count,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000
mean,3084.516306,13.061947,13.055774,13.057520,13.059868,13.281888,0.044124,0.033386,0.029391,0.032924,0.035678,103918.850254
std,812.408222,2.740808,2.741569,2.741426,2.741504,5.167194,0.014920,0.009898,0.011585,0.014435,0.016749,82820.967377
min,1868.581480,12.163000,12.269000,12.238000,12.204000,12.215000,0.019000,0.017000,0.016000,0.017000,0.017000,56.000000
25%,2265.567667,12.497250,12.503250,12.506000,12.505000,12.505500,0.034000,0.028000,0.023000,0.025000,0.027000,32868.250000
50%,2932.675390,12.631500,12.614500,12.606000,12.612000,12.622500,0.042000,0.032000,0.027000,0.030000,0.032000,76908.500000
75%,3706.901230,12.756750,12.746000,12.742000,12.741750,12.740750,0.051000,0.037000,0.033000,0.038000,0.041000,168883.250000
max,4507.519060,29.999000,29.999000,29.999000,29.999000,99.999000,0.105000,0.101000,0.140000,0.197000,0.233000,256102.000000


## Manejo del dataset

In [14]:
# Funcion que obtiene los indices de los outliers
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr*1.5)
    upper_bound = quartile_3 + (iqr*1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))

In [15]:
def high_photometric_errors(data):
    mer_mean = np.mean(data)
    mer_std = np.std(data)
    error_limit = mer_mean + 3*mer_std
    return np.where((data >= error_limit))

In [16]:
def lineal_fit(t,a,b):
    m = a+b*t
    return m

def parabolic_fit(t,a,b,c):
    m = a + b*t + c*t*t
    return m

def select_aperture(data):
    mean = []
    apertures = ["0","1","2","3","4"]
    for i in apertures:
        m = data["MER_"+i].mean()
        mean.append(m.values[0])
    values = mean.index(min(mean)) 
    return apertures[values]

# se deben actualizar las celdas posteriores a esta con estas funciones

# funcion que hace todo el preprocesamiento
def preprocessing(data,aperture):
    # 1 - se eliminan las mediciones con alto error fotometrico
    hpe_index = high_photometric_errors(data["MER_"+aperture].values)
    data = data.drop(data.index[hpe_index[0]])
    #2 - se eliminan datos atípicos
    outliers_index = outliers_iqr(data["MAG_"+aperture].values)
    data = data.drop(data.index[outliers_index[0]])
    # Se retornan los dias julianos y la magnitud de la apertura seleccionada
    return data["HJD"].values.ravel(),data["MAG_"+aperture].values.ravel()

# funcion que entrega los estadisticos propuestos por J_Godoy 
# Por ahora fit puede ser lineal o parabolico
def get_statistics(t,y):
    # Desviacion estandar y
    dep = np.std(yy)
    #para fit lineal
    poptl, pcovl = curve_fit(lineal_fit, t, y)
    y_hatl = lineal_fit(t,*poptl)
    perrl = np.sqrt(np.diag(pcovl))
    defl = np.sqrt(np.sum((y_hatl- y)*(y_hatl- y))/len(y))
    q1 = poptl[1]/perrl[1]
    c1 = 1-(defl/dep)
    # Para fit parabolico
    poptp, pcovp = curve_fit(parabolic_fit, t, y)
    y_hatp = parabolic_fit(t,*poptp)
    perrp = np.sqrt(np.diag(pcovp))
    defp = np.sqrt(np.sum((y_hatp- y)*(y_hatp- y))/len(y))
    q2 = poptp[2]/perrp[2]
    c2 = 1-(defp/defl)
    stat = [q1,c1,q2,c2]
    return stat

def get_ra_dec(data):
    ra= data.iloc[0]["RA"].values[0]
    ra= ra.split(" ")
    dec= data.iloc[0]["DEC"].values[0]
    dec= dec.split(" ")
    return ra[0],dec[0]

# Feature extraction

In [17]:
############################ Celda DE Prueba ####################################
ap = select_aperture(dataset)
tt,yy = preprocessing(dataset,ap) 
stat = get_statistics(tt,yy)

ra,dec= get_ra_dec(dataset)
print("================================== ")
print("RA= ",ra)
print("DEC=", dec)
print("========== Estadisticos ========== ")
print("Q1' = ", stat[0])
print("C1 = ", stat[1])
print("Q2' = ", stat[2])
print("C2 = ", stat[3])


RA=  0.989914
DEC= -15.943898
========== Estadisticos ========== 
Q1' =  0.0379314852703
C1 =  1.89144224716e-06
Q2' =  1.66054752579
C2 =  0.00364673226024


Como referencia, y en una primera iteración se esta usando el metodo propuesto por Javiera Godoy, el cual consiste en la aplicación de un ajuste polinomial (Lineal y parabolico) usando minimos cuadrados a la nube de puntos de la curva de luz, donde además se aplican dos criterios para identificar estrellas candidatas a variar en el largo periodo.

Fit lineal:  $ m = (a_0 \pm e_{a0}) + (a_1 \pm e_{a1})*t $

Fit Parabolico: $ m = (b_0 \pm e_{b0} ) + (b_1 \pm e_{b1})*t + (b_2 \pm e_{b2})*t^2 $

* Crieterio 1: La pendiente debe superar varias veces su error.
    * $\displaystyle Q_1' = \frac{a_1}{e_{a1}} $ -> para fit lineal
    * $\displaystyle Q_2' = \frac{b_2}{e_{b2}} $ -> para fi parabolico
    
Cabe mencionar que en el informe de JGodoy se calculaba un coeficiente $Q = Q' - \overline{Q'}$, donde $\overline{Q'}$ corresponde al promedio de los $Q'$ del campo correspondiente. Para este caso se simplificara y se usara $Q'$ a secas, ya que es necesario tener conocimientos astronomicos para saber a que campo pertenece cada estrella y cuantas estrellas pertenecen a dicho campo.

* Criterio 2: En caso de que exista variabilidad lineal o parabolica real, la dispersión debe disminuir. Tenemos las siguientes componentes:
    * DEP = Desviación estandar de la magnitud una vez quitado los outliers.
    * DEFL = Desviación estandar del fit lineal.
    * DEFP = Desviación estandar del fit parabolico.
    * $\displaystyle \frac{DEFL}{DEP} \le 1 - C_1$ -> Criterio para fit lineal.
    * $\displaystyle \frac{DEFP}{DEFL} \le 1 - C_2$ -> Criterio para fit parabolico.

Otras tenicas posibles a usar pueden ser:

* Transformada de fourier.
* Periodograma LS.
* Epoch folding en conjunto con otras tecnicas como por ejemplo ANOVA.

Sin embargo estas por el momento no se consideran ya que generalmente son utilizadas para encontrar periodos y lo que se busca es ver si efectivamente hay una variacion pequeña en la magnitud de la estrella en el largo periodo.

De manera provisoria se quitan los outliers usando el IQR. Otras formas de filtrar la data podria ser usando el parametro GRADE, o según el error fotometrico.

### Pruebas con todos los ejemplos

In [18]:
listado_test= open("ejemplos/lista.txt","r")
#listado_test = open("/home/horthp/Documentos/asas_data/all_data/test","r")
files_test = listado_test.readlines()
listado_test.close()

In [19]:
############################ Celda DE Prueba ####################################

# Esta celda demora en ejecutarse
col= ["ASAS NAME","RA","DEC","Q1","C1","Q2","C2"]
row= []
for i in files_test:
    archivo_test = open("/home/horthp/Documentos/Codigo_memoria/ejemplos/"+i.strip("\n"),"r")
    #archivo_test = open("/home/horthp/Documentos/asas_data/all_data/"+i.strip("\n"),"r")
    text_test = archivo_test.readlines()
    archivo_test.close()
    try:
        dataset = parser(text_test)
        ra,dec= get_ra_dec(dataset)
        ap = select_aperture(dataset)
        tt,yy = preprocessing(dataset,ap) 
        stat = get_statistics(tt,yy)
        row.append([i.strip("\n"),ra,dec,stat[0],stat[1],stat[2],stat[3]])
        # En caso que algún archivo tenga un error se guarda su nombre en otro archivo     
    except:
        archivo_fail = open("fails", "a")
        archivo_fail.write(i)
        archivo_fail.close()


In [20]:
df = pd.DataFrame(row,columns=col)

In [21]:
df.head()

,ASAS NAME,RA,DEC,Q1,C1,Q2,C2
0,005924-1556.6,0.989914,-15.943898,0.037931,0.000002,1.660548,0.003647
1,054902-5043.8,5.817406,-50.729909,1.780492,0.003106,-3.247457,0.010241
2,060523-0700.1,6.089784,-7.001341,-1.311879,0.001578,-1.305974,0.001567
3,072159-6604.2,7.366842,-66.071462,0.641543,0.000199,-3.303217,0.005230
4,072514-2748.0,7.420522,-27.799169,1.216728,0.001513,0.111487,0.000013


In [22]:
# Se exporta el dataframe obtenido
df.to_csv("estadisticos_all_test", sep=',', encoding='utf-8',index=False)

In [27]:
# funcion que obtiene los indices de las estrelllas que cumplen con los criterios
def index_criterion(data,criterion):
    if criterion == "lineal":
        selected = np.where((np.abs(data["Q1"]) > 4 ) & (np.abs(data["C1"]) > 0.02)
                           & ~(np.abs(data["Q2"]) > 4))
    if criterion == "parabolic":
        selected = np.where((np.abs(data["Q2"]) > 4 ) & (np.abs(data["C2"]) > 0.02)
                           & ~(np.abs(data["Q1"]) > 4))
    if criterion == "mix":
        selected = np.where((np.abs(data["Q1"]) > 4 ) & (np.abs(data["C1"]) > 0.02) 
                            & (np.abs(data["Q2"]) > 4 ) & (np.abs(data["C2"]) > 0.02))
    return selected

def stars_selection(data):

    # Se obtienen los indices de las estrellas que cumplen con los criterios
    lineal = index_criterion(data,"lineal")
    parabolic = index_criterion(data,"parabolic")
    mix = index_criterion(data,"mix")
    # Se obtienen todas las estrellas que cumplen con los criterios
    df_lineal = data.iloc[lineal]
    df_parabolic = data.iloc[parabolic]
    df_mix = data.iloc[mix]
    dataset = pd.concat([df_lineal,df_parabolic,df_mix],ignore_index=True)
    return dataset

selected_data= stars_selection(df)
selected_data.to_csv("selected_data", sep=',', encoding='utf-8',index=False)

In [28]:
selected_data

,ASAS NAME,RA,DEC,Q1,C1,Q2,C2
0,111905-8238.5,11.317693,-82.642158,-7.143861,0.021714,-2.173106,0.002072
1,144908-1150.8,14.818853,-11.846552,4.500165,0.022596,0.648103,0.000486
2,163624-0622.0,16.606738,-6.365813,11.036040,0.136505,-0.976079,0.001335
3,180525-2758.5,18.090277,-27.973530,6.591955,0.038566,1.586765,0.002367
4,201513-0113.7,20.253659,-1.228550,5.778267,0.055871,-1.276983,0.002973
5,190507-1517.9,19.085140,-15.298387,-2.074967,0.005517,-4.768174,0.028210
6,175619-3348.7,17.938513,-33.812124,10.129148,0.059226,-17.405903,0.149970


Según el documento de JGodoy las estrellas candidatas a tener una variabilidad ya sea lineal o parabolica en el largo periodo deben tener valores valores de $|Q_{1,2}| \ge 4$, $|C_{1,2}| \ge 0.02$